In [1]:
import random
import glob
import os

In [2]:

dataPath ='OneFold/data/'
trainPath = 'OneFold/Train/'
testPath = 'OneFold/Test/'
p=0.5

In [3]:
def clearFolder(path):
    for f in os.listdir(path):
        os.remove(os.path.join(path, f))

In [4]:
# Read txt file of symbols and change it to a dictionary of data which has key:position and value:symbol
#bold batch position by "-"
def data_to_dict(data):
    data = enumerate(data.strip().split(' '))
    data_dict = dict((i, j) for i, j in data)
    return data_dict

In [5]:
def read_data(file):
    with open(file, "r") as f:
        data=f.read().replace(" | ", " ")
        data = data.split('\n')
        data = [line for line in data if line.strip() != ""]
    return data

In [6]:
def get_key(val,my_dict):
    keys=list()
    for key, value in my_dict.items():
         if val == value:
              keys.append(key)
#     print(keys)
    return keys

In [7]:
def write_text(data,path):
    with open(path, "w+", encoding="utf-8") as fo:
        for d in data:
            d=list(d.values())
            fo.write(" ".join(str(i) for i in d))
            fo.write('\n')

In [8]:
def spilit_segments_of_chunks(file,chunks,k,p,l):
#     print(file)
    intersection_prc={}
    test_dict = dict()
    train_dict = dict()
    for chunk in chunks:
#         print(chunk)
        symbols=list(chunk.keys())
        segments_keys= [symbols[i:i + k] for i in range(0, len(symbols), k)]

        for j in segments_keys:
            rndm = random.random()
            d = {i: chunk[i] for i in j}
            if rndm >= p:
                test_dict.update(d)
                # test_count += 1
            else:
                train_dict.update(d)
        segments_keys=list()
    prc=len(set(test_dict.keys()).intersection(set(train_dict.keys())))/l
    intersection_prc[file]=prc
#     print(len(train_dict))
#     print(len(test_dict))
#     print("---------------")
    return test_dict,train_dict

    

In [9]:
# Chunks is list of n chunks of data: each chunk is a dictionary of key:position and value:symbol
# Chunks=[{},{},...,{}]
def chunke_data(data,keys,k,file):
#     print(data)
#     print(type(data))
    chunks=[]
    pre_key=0
    for key in keys:
        if(pre_key==0):
            chunks.append({key: data[key] for key in range(pre_key, min(key, len(data)))})
        else:
            temp=dict()
            temp={key: data[key] for key in range(max(0,pre_key), min(key, len(data)))}
            del temp[pre_key]
            chunks.append(temp)
        pre_key=key
    temp=dict()
#     temp={key: data[key] for key in range(max(0,pre_key), len(data))}
#     del temp[pre_key]
    chunks.append(temp)

    return chunks

In [10]:
def run(folder,k,p,train_folder,test_folder):
    for file in glob.glob(folder+"*.txt"):
        train=[]
        test=[]
        data=read_data(file)
        for line in data:
            d=data_to_dict(line)
#             keys=get_key("|",d)
#             print(keys)
            keys=[10000000000000000000000000000]
            d_test,d_train=spilit_segments_of_chunks(file,chunke_data(d,keys,k,file), k, p,len(d))
            test.append(d_test)
            train.append(d_train)
        write_text(test,test_folder+os.path.basename(file))
        write_text(train,train_folder+os.path.basename(file))
#         write_text(test,test_folder1+str(file))
#         write_text(train,train_folder1+str(file))


In [11]:
clearFolder(trainPath)
clearFolder(testPath)
intersection_prc={}
run(dataPath,3,p,trainPath,testPath)